In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key  = os.getenv('OPENAI_KEY')

In [3]:
def get_completion(prompt, temperature = 0.0, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    print(response)
    return response.choices[0].message["content"]


In [ ]:
import random
cars = [
    "2014 Toyota Camry",
    "2019 Ram 1500",
    "2020 Mazda Miata"
]
numerals = ["2", "3", "4"]
pets = ["no pets", f"""{random.choice(numerals)} {random.choice(["dogs", "cats", "pet pytons"])}"""]
family = ["single", f"""a single mom of {random.choice(numerals)}""", f"""a mother of {random.choice(numerals)}""", f"""a single father of {random.choice(numerals)}""", f"""a father of {random.choice(numerals)}"""]
unprofessionalOptions = ["Give your address and phone number in case anyone wants to contact you. ", "Use unprofessional tone and some slurs in your review. ", "Mention your personal business by name. ", "Mention how you bought your car and describe your interactions with the sales staff. ", "Mention someone at the dealership whose ethnic origin you did not care for. ", "Mention how your car was serviced by an incompetent mechanic. ", "Mention what condition the car was in when you bought it and all defects present. "]
familialStatus = random.choice(family)
petStatus = random.choice(pets)
car = random.choice(cars)
unprofessionalStatus = random.sample(unprofessionalOptions, random.choice(range(0, len(unprofessionalOptions))))


reviewPrompt = f"""
Write a car review of {car}. You are {familialStatus}. You have {petStatus}. {''.join(unprofessionalStatus)}
"""

print (reviewPrompt)
reviewResponse = get_completion(reviewPrompt, 1.2)

moderationPrompt = f"""
You are an online content moderation expert whose purpose is to classify vehicle reviews. Classification is either "Accepted" or "Rejected."
A review can get rejected if it contains inappropriate language, including profanities or slurs. In this case, the rejection reason is: "profanity." A review will also get rejected if it contains an email address, a phone number, or a URL. In this case, rejection reason is: "personally identifiable information." Another reason for rejection is if the review is focusing on the condition of the vehicle, for instance, that the seats are worn out, the tires are brand new, or the paint is scratched. Such characteristics are unlikely to help people looking for the same make and model vehicle and should be rejected as "vehicle specific." If the reviewer veers off topic too much and does not focus on the make and model of the car that is being reviewed, the review will also be rejected, with the reason "off topic." Finally, if the reviewer describes their experience at the car dealership, whether it is the sales process or service performed on the car, the review is also rejected, the reason being: "dealership." In all other cases, the review is accepted. If possible, give a few word snippet of offending text that caused the review to be rejected. Now, classify a vehicle review below delimited by triple backticks.
```{reviewResponse}```
First, summarize the review. Then, for each possible rejection reason, explain whether it applies and why. Explain your decision, even if the reason does not apply. Finally, output a json, with fields: classification, rejection_reason, offending_text. List every possible rejection reason and whether it is present or not (indicated by 1 or 0), along with a small sample of offending text for each reason.
"""

response = get_completion(moderationPrompt)
